多模态混合编码器-解码器（MED）

In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel, BertConfig, ViTModel

class BLIPModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config 
        # 图像编码器(ViT)
        self.vision_encoder = ViTModel(config.vit_config)
        # 文本编码器(BERT)
        self.text_encoder = BertModel(config.bert_config)
        # 文本解码器(带跨注意力的因果 BERT)
        decoder_config = BertConfig(**config.text_config.to_dict())
        decoder_config.is_decoder = True
        decoder_config.add_cross_attention = True
        self.text_decoder = BertModel(decoder_config)

        # 共享参数策略(除自注意力层外)
        self.__tie_weights_except_attn()
    
        # 投影层(用于对比学习)
        self.vision_proj = nn.Linear(config.vision_config.hidden_size, 256)
        self.text_proj = nn.Linear(config.text_config.hidden_size, 256)

        # ITM 分类头
        self.itm_head = nn.Linear(config.text_config.hidden_size * 2, 2)

    def _tie_weights_except_attn(self):
        """共享编码器/解码器的非注意力层参数"""
        # 共享嵌入层
        self.text_decoder.embeddings = self.text_encoder.embeddings

        # 共享 FFN 层（中间层和输出层）
        for i in range(self.config.num_layers):
             self.text_decoder.encoder.layer[i].intermediate = self.text_encoder.encoder.layer[i].intermediate
             self.text_decoder.encoder.layer[i].output = self.text_encoder.encoder.layer[i].output
